In [ ]:
# Import all the components
import os
import time
import json
from typing import Dict, Any

# Import the monitoring system
from quarkle_monitoring import (
    QuarkleMonitoring,
    ErrorTracker, 
    SlackNotifier,
    QuarkleCache,
    MemoryCache,
    track_errors
)

print("✅ All imports successful!")
print("📦 Available components:")
print("  - QuarkleMonitoring (orchestrator)")
print("  - ErrorTracker (error tracking)")
print("  - SlackNotifier (Slack alerts)")
print("  - QuarkleCache (Redis cache)")
print("  - MemoryCache (in-memory cache)")
print("  - track_errors (decorator)")


In [ ]:
# Setup the monitoring system (simple approach)
# Using memory cache for this demo (no Redis needed)

# MANUAL INPUT: Add your Slack token here
SLACK_TOKEN = ""  # <- Enter your Slack bot token here
SLACK_CHANNEL = "#alerts"  # <- Change to your channel

# Create the monitoring system
monitoring = QuarkleMonitoring(
    service_name="test-notebook",
    slack_token=SLACK_TOKEN if SLACK_TOKEN else None,
    slack_channel=SLACK_CHANNEL,
    use_redis=False,  # Use memory cache for demo
    version="notebook-v1.0"
)

print("🚀 Monitoring system initialized!")
print(f"📧 Service: {monitoring.service_name}")
print(f"💾 Cache type: {type(monitoring.cache).__name__}")
print(f"🔔 Slack configured: {monitoring.slack.client is not None}")

# Show what components are available
print("\n📦 Available via properties:")
print(f"  - monitoring.cache: {type(monitoring.cache).__name__}")
print(f"  - monitoring.error_tracker: {type(monitoring.error_tracker).__name__}")
print(f"  - monitoring.slack: {type(monitoring.slack).__name__}")


In [ ]:
# Test error tracking (will send Slack alert if token is provided)
print("🧪 Testing error tracking...")

# Track a 404 error
result = monitoring.track_error(
    error_code=404,
    endpoint="/api/users/123",
    user_id="test_user",
    extra={"request_id": "abc-123", "method": "GET"}
)

print(f"📊 Error tracked: {result}")

# Track a 403 error  
result2 = monitoring.track_error(
    error_code=403,
    endpoint="/api/admin",
    user_id="unauthorized_user",
    extra={"attempted_action": "delete_user"}
)

print(f"📊 Error tracked: {result2}")

# Try to track the same 404 again (should be rate limited)
result3 = monitoring.track_error(
    error_code=404,
    endpoint="/api/users/123",
    user_id="test_user"
)

print(f"📊 Error tracked (should be rate limited): {result3}")

# Track a 500 error (should be ignored - only 4xx errors are tracked)
result4 = monitoring.track_error(
    error_code=500,
    endpoint="/api/broken"
)

print(f"📊 500 error (ignored): {result4}")


In [ ]:
# Test lifecycle alerts (startup/shutdown notifications)
print("🔔 Testing lifecycle alerts...")

# Send startup alert
startup_result = monitoring.send_lifecycle_alert("startup", {
    "environment": "notebook",
    "python_version": "3.10",
    "startup_time": time.time()
})

print(f"🚀 Startup alert sent: {startup_result}")

# Send custom alert
custom_result = monitoring.send_lifecycle_alert("deployment", {
    "version": "v2.1.0",
    "deployed_by": "notebook_user",
    "features": ["new_error_tracking", "slack_integration"]
})

print(f"📦 Deployment alert sent: {custom_result}")

# Send shutdown alert
shutdown_result = monitoring.send_lifecycle_alert("shutdown", {
    "uptime_seconds": 300,
    "reason": "notebook_restart"
})

print(f"🛑 Shutdown alert sent: {shutdown_result}")


In [ ]:
# Test the decorator pattern
print("🎯 Testing decorator pattern...")

# Create a decorator using our error tracker
@track_errors(monitoring.error_tracker)
def api_endpoint(user_id: str, action: str):
    """Simulate an API endpoint that might fail."""
    if action == "forbidden":
        error = Exception("Access denied")
        error.status_code = 403
        raise error
    elif action == "not_found":
        error = Exception("Resource not found")
        error.status_code = 404
        raise error
    elif action == "server_error":
        error = Exception("Internal server error")
        error.status_code = 500
        raise error
    else:
        return f"Success: {action} for user {user_id}"

# Test successful execution
try:
    result = api_endpoint("user123", "get_profile")
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"❌ Error: {e}")

# Test 403 error (should be tracked and alert sent)
try:
    result = api_endpoint("user456", "forbidden")
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"🔒 403 Error tracked: {e}")

# Test 404 error (should be tracked and alert sent)
try:
    result = api_endpoint("user789", "not_found")
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"🔍 404 Error tracked: {e}")

# Test 500 error (should NOT be tracked - only 4xx errors)
try:
    result = api_endpoint("user999", "server_error")
    print(f"✅ Success: {result}")
except Exception as e:
    print(f"💥 500 Error (not tracked): {e}")


In [ ]:
# Advanced: Use individual components
print("🔧 Testing individual components...")

# Option 1: Use Redis cache (requires AWS/Redis setup)
print("\n1️⃣ Redis Cache Setup:")
try:
    redis_cache = QuarkleCache(stage="dev", service_name="notebook-test")
    print(f"   ✅ Redis cache initialized: {redis_cache._client is not None}")
except Exception as e:
    print(f"   ❌ Redis failed (expected): {e}")
    redis_cache = None

# Option 2: Use memory cache (always works)
print("\n2️⃣ Memory Cache Setup:")
memory_cache = MemoryCache()
print(f"   ✅ Memory cache initialized")

# Test cache operations
print("\n3️⃣ Cache Operations:")
cache_to_test = redis_cache if redis_cache and redis_cache._client else memory_cache
print(f"   Using: {type(cache_to_test).__name__}")

# Test rate limiting
allowed1 = cache_to_test.rate_limit("test_action", 1, 60)
print(f"   First rate limit check: {allowed1}")

allowed2 = cache_to_test.rate_limit("test_action", 1, 60)  
print(f"   Second rate limit check (should be False): {allowed2}")

# Test set/get if supported
if hasattr(cache_to_test, 'get'):
    cache_to_test.set("test_key", "test_value", 30)
    value = cache_to_test.get("test_key")
    print(f"   Set/Get test: {value}")

print("\n4️⃣ Independent Error Tracker:")
independent_tracker = ErrorTracker(
    cache=cache_to_test,
    service_name="independent-service",
    version="manual-v1.0"
)

should_alert, error_data = independent_tracker.track_error(
    error_code=401,
    endpoint="/api/auth",
    user_id="test_user_2",
    extra={"auth_method": "oauth"}
)

print(f"   Independent error tracking: {should_alert}")
if error_data:
    print(f"   Error hash: {error_data.get('error_hash', 'N/A')}")
    print(f"   Service: {error_data.get('service', 'N/A')}")
    print(f"   Version: {error_data.get('git_hash', 'N/A')}")

print("\n5️⃣ Independent Slack Notifier:")
independent_slack = SlackNotifier(
    token=SLACK_TOKEN if SLACK_TOKEN else None,
    channel=SLACK_CHANNEL
)

if should_alert and error_data and independent_slack.client:
    try:
        slack_result = independent_slack.send_error_alert(error_data)
        print(f"   ✅ Slack alert sent independently: {slack_result}")
    except Exception as e:
        print(f"   ❌ Slack alert failed: {e}")
else:
    print(f"   ⏭️  Slack alert skipped (token: {SLACK_TOKEN is not None and SLACK_TOKEN != ''}, should_alert: {should_alert})")


In [ ]:
# Demonstrate protocol-based design
print("🔌 Protocol-Based Design Demo")

# Create a custom cache that implements the protocol
class CustomCache:
    """Custom cache that implements CacheInterface protocol."""
    
    def __init__(self):
        self.data = {}
        self.rate_limits = {}
        print("   📦 CustomCache initialized")
    
    def rate_limit(self, key: str, limit: int, window: int = 3600) -> bool:
        """Simple rate limiting implementation."""
        import time
        now = time.time()
        
        if key in self.rate_limits:
            last_time = self.rate_limits[key]
            if now - last_time < window:
                return False  # Rate limited
        
        self.rate_limits[key] = now
        return True  # Allowed
    
    def set(self, key: str, value: str, ttl: int = 3600) -> bool:
        """Store data (ignoring TTL for simplicity)."""
        self.data[key] = value
        return True

# Use custom cache with ErrorTracker (no inheritance needed!)
print("\n🔗 Using custom cache with ErrorTracker:")
custom_cache = CustomCache()

# This works because CustomCache implements the required methods
protocol_tracker = ErrorTracker(
    cache=custom_cache,  # Duck typing - no inheritance required!
    service_name="protocol-demo",
    version="custom-v1.0"
)

print("   ✅ ErrorTracker accepts custom cache (protocol-based)")

# Test it works
should_alert, error_data = protocol_tracker.track_error(
    error_code=422,
    endpoint="/api/validation",
    extra={"field": "email", "reason": "invalid_format"}
)

print(f"   📊 Custom cache error tracking: {should_alert}")
print(f"   🎯 Error hash: {error_data.get('error_hash', 'N/A') if error_data else 'N/A'}")

# Show that the data was stored in our custom cache
print(f"   💾 Custom cache now has {len(custom_cache.data)} items stored")
print(f"   ⏱️  Rate limits tracked: {len(custom_cache.rate_limits)} keys")

print("\n✨ This demonstrates the power of Protocol-based design:")
print("   - No inheritance required")
print("   - Any class with rate_limit() and set() methods works")
print("   - Enables 'duck typing' with type hints")
